In [23]:
import requests
import pandas as pd

API_KEY = "YOUR_API_KEY"

# Function to get places within a radius from a given lat/lng for multiple place types
def get_places_nearby_multiple_types(lat, lng, radius=1609, place_types=None):
    """
    Get places of multiple types within a radius from a given lat/lng.
    :param lat: Latitude of the dropped pin
    :param lng: Longitude of the dropped pin
    :param radius: Radius in meters (default is 1 mile which is ~1609 meters)
    :param place_types: List of place types to search (e.g., ['business', 'restaurant', etc.])
    :return: List of places with details
    """
    if place_types is None:
        place_types = ['business']  # Default place type if none provided

    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    all_places = []

    for place_type in place_types:
        params = {
            'location': f'{lat},{lng}',
            'radius': radius,
            'type': place_type,
            'key': API_KEY
        }

        while True:
            response = requests.get(endpoint_url, params=params)
            if response.status_code == 200:
                places_data = response.json()
                if 'results' in places_data:
                    for place in places_data['results']:
                        place_info = {
                            'name': place.get('name'),
                            'address': place.get('vicinity'),
                            'location': place.get('geometry', {}).get('location'),
                            'rating': place.get('rating', 'N/A'),
                            'user_ratings_total': place.get('user_ratings_total', 'N/A'),
                            'place_id': place.get('place_id'),
                            'type': place_type
                        }
                        all_places.append(place_info)
                if 'next_page_token' in places_data:
                    params['pagetoken'] = places_data['next_page_token']
                    # Google API requires a short delay before using the next_page_token
                    import time
                    time.sleep(2)
                else:
                    break
            else:
                print(f"Error for place type '{place_type}': {response.status_code}")
                break

    return all_places

# Function to get all possible place types supported by the Google Places API
def get_all_possible_place_types():
    return [
        "accounting", "airport", "amusement_park", "aquarium", "art_gallery", "atm", "bakery", "bank", "bar",
        "beauty_salon", "bicycle_store", "book_store", "bowling_alley", "bus_station", "cafe", "campground", "car_dealer",
        "car_rental", "car_repair", "car_wash", "casino", "cemetery", "church", "city_hall", "clothing_store",
        "convenience_store", "courthouse", "dentist", "department_store", "doctor", "electrician", "electronics_store",
        "embassy", "fire_station", "florist", "funeral_home", "furniture_store", "gas_station", "gym", "hair_care",
        "hardware_store", "hindu_temple", "home_goods_store", "hospital", "insurance_agency", "jewelry_store", "laundry",
        "lawyer", "library", "light_rail_station", "liquor_store", "local_government_office", "locksmith", "lodging",
        "meal_delivery", "meal_takeaway", "mosque", "movie_rental", "movie_theater", "moving_company", "museum",
        "night_club", "painter", "park", "parking", "pet_store", "pharmacy", "physiotherapist", "plumber", "police",
        "post_office", "primary_school", "real_estate_agency", "restaurant", "roofing_contractor", "rv_park",
        "school", "secondary_school", "shoe_store", "shopping_mall", "spa", "stadium", "storage", "store",
        "subway_station", "supermarket", "synagogue", "taxi_stand", "tourist_attraction", "train_station",
        "transit_station", "travel_agency", "university", "veterinary_care", "zoo"
    ]



In [24]:
# Example usage: Replace lat, lng with your dropped pin coordinates
latitude = 40.748817  # Example: Latitude of the dropped pin (e.g., Empire State Building)
longitude = -73.985428  # Example: Longitude of the dropped pin
radius_meters = 750  # 1 mile = 1609 meters
place_types = ['restaurant', 'park', 'cafe']  # You can add multiple place types

places = get_places_nearby_multiple_types(latitude, longitude, radius_meters, place_types)

In [ ]:
places_df = pd.DataFrame(places)
places_df

In [ ]:
places_df.to_csv('places.csv', index=False)